In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd

In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2")

d:\Anaconda\envs\nightshade\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ICtSP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Test

In [3]:
sentences = [
    "I love you today",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

(3, 384)
tensor([[1.0000, 0.2323, 0.1421],
        [0.2323, 1.0000, 0.1411],
        [0.1421, 0.1411, 1.0000]])


d:\Anaconda\envs\nightshade\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
input_csv_file_path = 'drug_interactions.csv'
output_csv_file_path = 'test.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file_path)

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Define the path to your input and output CSV files
input_csv_file_path = 'cluster.csv'
output_csv_file_path = 'transformed_drug_interactions.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file_path)

# Check if the DataFrame is empty
if df.empty:
    print("The input CSV file is empty.")
else:
    # Get the last column name
    last_column_name = df.columns[-1]

    # Initialize the transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can change the model to another pre-trained one

    # Iterate over each row in the last column, encode the text, and append to the DataFrame
    for index, row in df.iterrows():
        # Extract the text from the last column for the current row
        text = str(row[last_column_name])

        # Encode the text using the transformer model
        embedding = model.encode(text)

        # Add each embedding dimension as a new column for the current row
        for i, value in enumerate(embedding):
            df.at[index, f'{last_column_name}_dim_{i}'] = value

    # Save the updated DataFrame with the appended embeddings to a new CSV file
    df.to_csv(output_csv_file_path, index=False)

    print(f"Transformed data saved to {output_csv_file_path}")


d:\Anaconda\envs\nightshade\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\Anaconda\envs\nightshade\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\ICtSP\AppData\Local\Temp\ipykernel_5348\1349754658.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.

Transformed data saved to transformed_drug_interactions.csv


Cluster

In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

df = pd.read_csv("transformed_drug_interactions.csv")

# Step 1: Extract the transformer output columns
transformer_columns = [col for col in df.columns if col.startswith('description_modified_dim_')]
transformer_data = df[transformer_columns]

# Step 2: Normalize the data
scaler = StandardScaler()
transformer_data_scaled = scaler.fit_transform(transformer_data)

# Step 3: Apply K-Means clustering
kmeans = KMeans(n_clusters=20, random_state=42)  # You can change the number of clusters (n_clusters) as needed
df['cluster'] = kmeans.fit_predict(transformer_data_scaled)

# Save the clustered data to a new CSV file
output_file_path = 'clustered_drug_interactions.csv'
df.to_csv(output_file_path, index=False)

output_file_path

'clustered_drug_interactions.csv'

Only keep the sentence part, remove the vector

In [11]:
output_file_first_6_cols = 'first_6_columns.csv'
df = pd.read_csv("clustered_drug_interactions.csv")
# Select the first 6 columns
first_6_columns = df.iloc[:, :6]

# Save the selected columns to a new CSV file
first_6_columns.to_csv(output_file_first_6_cols, index=False)

print(f"File saved to: {output_file_first_6_cols}")


File saved to: first_6_columns.csv


Use A and B to replace

In [13]:
cluster_df = pd.read_csv("cluster.csv")
cluster_df['description_modified'] = cluster_df.apply(
    lambda row: row['description'].replace(row['drug-name'], 'A').replace(row['drug-interact-name'], 'B'), axis=1
)

# Save the modified DataFrame to a new CSV file
output_file_path_modified = 'cluster_description.csv'
cluster_df.to_csv(output_file_path_modified, index=False)

print(f"Modified file saved to: {output_file_path_modified}")

Modified file saved to: cluster_description.csv


In [5]:
import pandas as pd

# Load the CSV file
file_path = 'clustered_drug_interactions.csv'
data = pd.read_csv(file_path)

# Group the data by cluster and get the 'description_modified' for each cluster
clustered_descriptions = data.groupby('cluster')['description_modified'].apply(lambda x: list(set(x))).reset_index()

# Define the maximum number of columns to display
max_cols = 10  # You can adjust this number as needed

# Expand the 'description_modified' list into separate columns
clustered_descriptions_expanded = clustered_descriptions.copy()
clustered_descriptions_expanded[['desc_' + str(i + 1) for i in range(max_cols)]] = pd.DataFrame(
    clustered_descriptions['description_modified'].apply(lambda x: x[:max_cols]).to_list(), index=clustered_descriptions.index)

# Drop the original 'description_modified' column for better clarity
clustered_descriptions_expanded = clustered_descriptions_expanded.drop(columns=['description_modified'])

# Save the expanded dataframe to a CSV file
output_file_path = 'clustered_descriptions_expanded.csv'  # Replace with your desired output file path
clustered_descriptions_expanded.to_csv(output_file_path, index=False)

print(f"The CSV file has been saved to {output_file_path}")

PermissionError: [Errno 13] Permission denied: 'clustered_descriptions_expanded.csv'